In [33]:
from transformers import BertTokenizer, BertModel, DistilBertTokenizer, DistilBertModel
from torch.utils.data import DataLoader, TensorDataset
import torch

# Load pre-trained model tokenizer (vocabulary)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Encode the input text


In [34]:
import pandas as pd
file_path = 'processed_reddits.csv'
df = pd.read_csv(file_path)

In [35]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
encoded_inputs = tokenizer(df['processed'].tolist(), padding=True, truncation=True, return_tensors='pt')
input_ids = encoded_inputs['input_ids']
attention_mask = encoded_inputs['attention_mask']
dataset = TensorDataset(input_ids, attention_mask)
dataloader = DataLoader(dataset, batch_size=16)

In [36]:
embeddings_list = [] 
def get_embeddings(dataloader, model, device):
    model.eval()  # Set model to evaluation mode
    all_embeddings = []

    with torch.no_grad():  # Disable gradient calculation
        for batch in dataloader:
            input_ids, attention_mask = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            last_hidden_states = outputs.last_hidden_state

            # Optionally, take the [CLS] token embedding
            cls_embeddings = last_hidden_states[:, 0, :].cpu().numpy()
            all_embeddings.extend(cls_embeddings)

    return all_embeddings
    


In [37]:
embeddings = get_embeddings(dataloader, model, device)

# Convert embeddings to a DataFrame
embeddings_df = pd.DataFrame(embeddings)

# Optionally, concatenate the embeddings with the original DataFrame
result_df = pd.concat([df, embeddings_df], axis=1)

# Print the result
print(result_df)

KeyboardInterrupt: 

In [ ]:
from datetime import datetime, timedelta
import numpy as np
start_date = df['date'].min()
end_date = df['date'].max()

In [ ]:
delta = timedelta(days=1)
embedding_dict = dict()
current_date = start_date
while current_date <= end_date:
    current_df = df[df['date'] == current_date]
    embedding_dict[current_date] = np.mean(current_df['embedding'], axis=0)
    current_date += delta